# Decompounded Options

# 👉 <a id = "top">Table of Contents</a> 👈 

# [1. Functions](#p1)

# [2. Question 1](#p2)

# [3. Question 2](#p3)

In [1]:
import pandas as pd
from scipy import integrate
from scipy.stats import norm
import numpy as np

# <a id = "p1">1.</a>  <font color = "green"> Functions </font>  [back to table of contents](#top)

In [2]:
def black76_Call(F, K, T, sigma):
    #discount_factor = np.exp(-r*T)
    #F = S0*np.exp(r*T)
    d1 = (np.log(F/K)+(1/2)*(sigma**2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    price = F*norm.cdf(d1) - K*norm.cdf(d2)
    return price

In [3]:
def black76_Put(F, K, T, sigma):
    #discount_factor = np.exp(-r*T)
    #F = S0*np.exp(r*T)
    d1 = (np.log(F/K)+1/2*(sigma**2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    price = K*norm.cdf(-d2) - F*norm.cdf(-d1)
    return price

In [ ]:
def SABR(F, 
         K, 
         T, 
         alpha, 
         beta, 
         rho, 
         nu):
    X = K
    # if K is at-the-money-forward
    # if abs(F - K) < 1e-12:
    if F == K:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        # if np.isnan(zhi):
        #     print((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        #     return
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

In [5]:
def IRR(K, tenor, delta):
    sum = 0
    for i in np.arange(1, tenor/delta + 1):
        sum += delta*(1 + K*delta)**(-i)
    return sum

# first derivative
def IRR1(K, tenor, delta):
    sum = 0
    for i in np.arange(1, tenor/delta + 1):
        sum += (-i)*(delta**2)*(1 + K*delta)**(-i-1)
    return sum

# second derivative
def IRR2(K, tenor, delta):
    sum = 0
    for i in np.arange(1, tenor/delta + 1):
        sum += (-i)*(-i-1)*(delta**3)*(1 + K*delta)**(-i-2)
    return sum

# <a id = "p2">2.</a>  <font color = "green"> Question 1 </font>  [back to table of contents](#top)

### Question 1   

#### Use static replication to value the PV of payoff: $CMS\ 10y^{1/4} - 0.04^{1/2}$ at time T = 5y

$g(F) = F^{\frac{1}{4}} - 0.2 \ \ \ \ \ \ g'(F) = \frac{1}{4}F^{-\frac{3}{4}} \ \ \ \ \ \ \ g''(F) = -\frac{3}{16}F^{-\frac{7}{4}}$

$ h(K) = \frac{g(K)}{IRR(K)}$ <br><br>
$ h'(K) = \frac{IRR(K)g'(K) - g(K)IRR'(K)}{IRR(K)^2}$<br><br>
$ h''(K) = \frac{IRR(K)g''(K) - IRR''(K)g(K) - 2 * IRR'(K)g'(K)}{IRR(K)^2} +\frac{2*IRR'(K)^2g(K)}{IRR(K)^3}$

In [6]:
def g(K):
    return K**0.25 - 0.2

def g1(K):
    return 0.25*K**(-0.75)

def g2(K):
    return (-3/16)*K**(-7/4)

In [7]:
def h(K, tenor, delta):
    return g(K)/IRR(K, tenor, delta)

def h1(K,  tenor, delta):
    nominator = IRR(K, tenor, delta)*g1(K) - g(K)*IRR1(K, tenor, delta)
    denominator = IRR(K, tenor, delta)**2
    return nominator/denominator

def h2(K, tenor, delta):
    nominator = IRR(K, tenor, delta)*g2(K) - g(K)*IRR2(K, tenor, delta) - 2*IRR1(K, tenor, delta)*g1(K)
    denominator = IRR(K, tenor, delta)**2
    term2 = 2*IRR1(K, tenor, delta)**2*g(K)/IRR(K, tenor, delta)**3
    return nominator/denominator + term2

$V_0 = D(0, T)g(F) + \int_0^F h''(K)V^{rec}(K)\,dK + \int_F^\infty h''(K)V^{pay}(K)\,dK$

In [ ]:
Discount_Factors = pd.read_csv('Discount_Factors.csv')      # This is from Part 1
Discount_Factors

,Years,Rate,OIS_Discount_Factor,IRS_Discount_Factor,Forward_LIBOR
0,0.5,0.025000,0.998752,0.987654,0.025000
1,1.0,0.028000,0.997009,0.972577,0.031245
2,1.5,0.029000,0.995270,0.957378,0.031242
3,2.0,0.030000,0.993531,0.942179,0.033275
4,2.5,0.030750,0.991773,0.926330,0.034038
5,3.0,0.031500,0.990015,0.910482,0.035565
6,3.5,0.032000,0.988066,0.894731,0.035309
7,4.0,0.032500,0.986117,0.878981,0.036328
8,4.5,0.032750,0.984150,0.863985,0.035053
9,5.0,0.033000,0.982184,0.848989,0.035562


In [ ]:
D = Discount_Factors[Discount_Factors['Years'] == 5]['OIS_Discount_Factor'].iloc[0]     # ois D(0,5)
D

0.9821841197332212

In [10]:
Forward_swap_rates = pd.read_csv('Forward_swap_rates_df.csv')      # This is from Part 1
Forward_swap_rates

,Start,Tenor,Forward_Swap_Rates
0,1,1,0.032007
1,1,2,0.033259
2,1,3,0.034011
3,1,5,0.035255
4,1,10,0.038428
5,5,1,0.039274
6,5,2,0.040075
7,5,3,0.040072
8,5,5,0.041093
9,5,10,0.043634


In [11]:
F = Forward_swap_rates[(Forward_swap_rates['Start']== 5) & (Forward_swap_rates['Tenor'] == 10)]["Forward_Swap_Rates"].iloc[0]
F

0.0436336455274827

In [ ]:
Calibration_parameters_df = pd.read_csv('Calibration_parameters_df.csv')
Calibration_parameters_df

,start,tenor,Beta,alpha,rho,nu,sigma
0,1,1,1.638113e-10,0.139074,-0.633223,2.049416,0.2250
1,1,2,1.016550e-07,0.184650,-0.525114,1.677383,0.2872
2,1,3,8.475424e-13,0.196851,-0.482848,1.438125,0.2978
3,1,5,8.330658e-11,0.178059,-0.414493,1.064842,0.2607
4,1,10,2.489366e-05,0.169984,-0.255690,0.792209,0.2447
5,5,1,8.974038e-07,0.166618,-0.585668,1.340427,0.2726
6,5,2,1.571376e-07,0.199534,-0.547030,1.062005,0.2983
7,5,3,1.579106e-05,0.210314,-0.549687,0.936773,0.2998
8,5,5,7.319718e-06,0.190244,-0.506814,0.674546,0.2660
9,5,10,5.778986e-02,0.174809,-0.415705,0.511310,0.2451


In [ ]:
alpha = Calibration_parameters_df[(Calibration_parameters_df['start']== 5) & (Calibration_parameters_df['tenor'] == 10)]["alpha"]
alpha

9    0.174809
Name: alpha, dtype: float64

In [ ]:
rho = Calibration_parameters_df[(Calibration_parameters_df['start']== 5) & (Calibration_parameters_df['tenor'] == 10)]["rho"]
rho

9   -0.415705
Name: rho, dtype: float64

In [ ]:
nu = Calibration_parameters_df[(Calibration_parameters_df['start']== 5) & (Calibration_parameters_df['tenor'] == 10)]["nu"]
nu

9    0.51131
Name: nu, dtype: float64

In [16]:
tenor = 10
delta = 0.5
T = 5
beta = 0.9

In [17]:
V_rec = integrate.quad(lambda K: h2(K, tenor, delta)*black76_Put(F, K, T, SABR(F, K, T, alpha, beta, rho, nu)), 0, F)
V_pay = integrate.quad(lambda K: h2(K, tenor, delta)*black76_Call(F, K, T, SABR(F, K, T, alpha, beta, rho, nu)), F, 1000)

pv = D* g(F) + V_rec[0] + V_pay[0]
pv

0.24976811691374118

# <a id = "p3">3.</a>  <font color = "green"> Question 2 </font>  [back to table of contents](#top)

#### Use static replication to value the PV of this payoff: $(CMS\ 10y^{1/4} - 0.04^{1/2})^+$

$F^\frac{1}{4} > 0.2$<br>
$F > 0.2^4$<br>
$F > 0.0016 = L$

$CMS\ Caplet = h'(L)V^{pay}(L) + \int^{\infty}_Lh''(K)V^{pay}(K)dK$

In [19]:
tenor = 10
delta = 2
T = 5
L = 0.2**4

term1 = h1(L, tenor, delta)*black76_Call(F, L, T, SABR(F, L, T, alpha, beta, rho, nu))
term2 = integrate.quad(lambda K: h2(K, tenor,delta)*black76_Call(F, K, T, SABR(F, K, T, alpha, beta, rho, nu)), L,1000)
PV_caplet = term1[0]+ term2[0]
PV_caplet

0.03207684703595258